In [98]:
import pandas as pd
import os
from skimage import io
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, classification_report, f1_score, roc_auc_score
import pickle
import Functions2 as ef
from sklearn.linear_model import LogisticRegression as LR
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, classification_report
import numpy as np
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score, accuracy_score
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)


**Bad data**

In [99]:
# Load the data
data_path = r'C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\full_data.csv'
df = pd.read_csv(data_path)

# Preprocess the diagnostic column
df['diagnostic'] = df['diagnostic'].map({'BCC': 1, 'MEL': 1, 'SCC': 1, 'ACK': 0, 'NEV': 0, 'SEK': 0})

# Define the function to extract features
def extract_features(folder_path):
    feature_1 = []
    feature_2 = []
    feature_3 = []
    feature_4 = []
    feature_5 = []
    feature_6 = []
    feature_7 = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            original = io.imread(image_path)

            # Ignore the alpha channel (e.g. transparency)
            if original.shape[-1] == 4:
                original = original[..., :3]

            feature_1.append(ef.measure_pigment_network(original))
            feature_2.append(ef.measure_blue_veil(original))
            feature_3.append(ef.measure_vascular(original))
            feature_4.append(ef.measure_globules(original))
            feature_5.append(ef.measure_streaks(original))
            feature_6.append(ef.measure_irregular_pigmentation(original))
            feature_7.append(ef.measure_regression(original))

    return feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7

# Define the folder path for image processing
folder_path_in = r'C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\ColorMask\Training'

# Extract features from the images
feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7 = extract_features(folder_path_in)

# Create a DataFrame for the features
features_df = pd.DataFrame()
features_df["img_id"] = [filename for filename in os.listdir(folder_path_in) if filename.endswith(('.jpg', '.png'))]
features_df["1: pigment network"] = feature_1
features_df["2: Blue veil"] = feature_2
features_df["3: Vascular"] = feature_3
features_df["4: Globules"] = feature_4
features_df["5: Streaks"] = feature_5
features_df["6: Pigmentation"] = feature_6
features_df["7: Regression"] = feature_7

# Merge the features DataFrame with the diagnostic column from the original DataFrame
df_merged2 = pd.merge(df[['img_id', 'diagnostic']], features_df, on='img_id', how='inner')

# Split the data into training and testing sets
X = df_merged2.drop(['img_id', 'diagnostic'], axis=1)
Y = df_merged2['diagnostic']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5)

classifiers = [LR(), KNN(), DTC()]  # Replace with your trained classifiers

# Perform cross-validation for each classifier
for classifier in classifiers:
    y_pred = cross_val_predict(classifier, X, Y, cv=5)  # Change cv value as per your requirement
    
    # Calculate evaluation metrics
    f1 = f1_score(Y, y_pred)
    precision = precision_score(Y, y_pred)
    recall = recall_score(Y, y_pred)
    tn, fp, fn, tp = confusion_matrix(Y, y_pred).ravel()
    specificity = tn / (tn + fp)
    
    print(f"Classifier: {type(classifier).__name__}")
    print(f"F1 score: {f1}")
    print(f"Precision: {precision}")
    print(f"Recall/Sensitivity: {recall}")
    print(f"Specificity: {specificity}")
    print("Confusion Matrix:")
    print(confusion_matrix(Y, y_pred))
    print()

Classifier: LogisticRegression
F1 score: 0.62
Precision: 0.6458333333333334
Recall/Sensitivity: 0.5961538461538461
Specificity: 0.6792452830188679
Confusion Matrix:
[[36 17]
 [21 31]]

Classifier: KNeighborsClassifier
F1 score: 0.6796116504854369
Precision: 0.6862745098039216
Recall/Sensitivity: 0.6730769230769231
Specificity: 0.6981132075471698
Confusion Matrix:
[[37 16]
 [17 35]]

Classifier: DecisionTreeClassifier
F1 score: 0.5825242718446602
Precision: 0.5882352941176471
Recall/Sensitivity: 0.5769230769230769
Specificity: 0.6037735849056604
Confusion Matrix:
[[32 21]
 [22 30]]



**KNN MODEL AND PICKLE**

In [100]:
import numpy as np
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score

# Create KNN model
knn_model = KNeighborsClassifier(n_neighbors=5)  # Specify the number of neighbors

# Train the KNN model on your data (X and Y)
knn_model.fit(X, Y)

# Pickle the KNN model
with open(r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\src\Workspace\CleanLast\knn_model.pkl", "wb") as f:
    pickle.dump(knn_model, f)

# Load the pickled KNN model
with open(r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\src\Workspace\CleanLast\knn_model.pkl", "rb") as f:
    loaded_knn_model = pickle.load(f)

# Perform cross-validation using the loaded KNN model
y_pred = cross_val_predict(loaded_knn_model, X, Y, cv=5)

# Calculate evaluation metrics
f1 = f1_score(Y, y_pred)
precision = precision_score(Y, y_pred)
recall = recall_score(Y, y_pred)
tn, fp, fn, tp = confusion_matrix(Y, y_pred).ravel()
specificity = tn / (tn + fp)

print("Loaded KNN Model Metrics:")
print(f"F1 score: {f1}")
print(f"Precision: {precision}")
print(f"Recall/Sensitivity: {recall}")
print(f"Specificity: {specificity}")
print("Confusion Matrix:")
print(confusion_matrix(Y, y_pred))

Loaded KNN Model Metrics:
F1 score: 0.6796116504854369
Precision: 0.6862745098039216
Recall/Sensitivity: 0.6730769230769231
Specificity: 0.6981132075471698
Confusion Matrix:
[[37 16]
 [17 35]]


**Predict with Knn**

In [101]:
import numpy as np
import pickle
import cv2
import os
import pandas as pd

# Load the pickled LR model
with open(r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\src\Workspace\CleanLast\knn_model.pkl", "rb") as f:
    model = pickle.load(f)

# Load the data
data_path = r'C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\full_data.csv'
df = pd.read_csv(data_path)

# Preprocess the diagnostic column
df['diagnostic'] = df['diagnostic'].map({'BCC': 1, 'MEL': 1, 'SCC': 1, 'ACK': 0, 'NEV': 0, 'SEK': 0})

# Define the function to extract features
def extract_features2(image_path):
    image = cv2.imread(image_path)
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # Perform feature extraction operations
    feature_1 = ef.measure_pigment_network(lab_image)
    feature_2 = ef.measure_blue_veil(lab_image)
    feature_3 = ef.measure_vascular(lab_image)
    feature_4 = ef.measure_globules(lab_image)
    feature_5 = ef.measure_streaks(lab_image)
    feature_6 = ef.measure_irregular_pigmentation(lab_image)
    feature_7 = ef.measure_regression(lab_image)

    return feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7

# Define the folder path for image processing
folder_path_in = r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\ColorMask\Test"  # Replace with the path to the folder containing images

# Iterate over images in the folder
for filename in os.listdir(folder_path_in):
    if filename.endswith(".jpg") or filename.endswith(".png"):  # Update the file extensions if necessary
        image_path = os.path.join(folder_path_in, filename)
        
        # Extract features from the image
        feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7 = extract_features2(image_path)
        
        # Prepare the data for prediction
        X = np.array([feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7]).reshape(1, -1)
        
        # Make predictions using the loaded LR model
        probabilities = model.predict_proba(X)
        
        # Get the probability for class 1 (positive class)
        probability_positive = probabilities[0][1]
        
        # Map the probability to a value between 0 and 1
        mapped_value = 1.0 - probability_positive  # Invert the probability to get the desired mapping

        # Get the diagnosis from the df DataFrame using the modified filename
        diagnosis = df.loc[df['img_id'] == filename, 'diagnostic'].values[0]

        # Map the diagnosis to a human-readable label
        diagnosis_label = 'Not Healthy' if diagnosis == 1 else 'Healthy'
        
        # Print the image filename, diagnosis, and mapped value
        print("Image:", filename)
        print("Diagnosis from csv:", diagnosis_label)
        print("Prediction [0:Healthy - 1:Not Healthy]:", mapped_value)
        print("---------------------")


Image: PAT_109_868_723.png
Diagnosis from csv: Not Healthy
Prediction [0:Healthy - 1:Not Healthy]: 1.0
---------------------
Image: PAT_236_361_180.png
Diagnosis from csv: Healthy
Prediction [0:Healthy - 1:Not Healthy]: 0.4
---------------------
Image: PAT_26_37_865.png
Diagnosis from csv: Healthy
Prediction [0:Healthy - 1:Not Healthy]: 0.8
---------------------
Image: PAT_270_417_257.png
Diagnosis from csv: Not Healthy
Prediction [0:Healthy - 1:Not Healthy]: 0.4
---------------------
Image: PAT_333_1469_499.png
Diagnosis from csv: Healthy
Prediction [0:Healthy - 1:Not Healthy]: 0.8
---------------------
Image: PAT_38_54_234.png
Diagnosis from csv: Not Healthy
Prediction [0:Healthy - 1:Not Healthy]: 0.8
---------------------
Image: PAT_398_799_163.png
Diagnosis from csv: Healthy
Prediction [0:Healthy - 1:Not Healthy]: 0.4
---------------------
Image: PAT_42_58_13.png
Diagnosis from csv: Not Healthy
Prediction [0:Healthy - 1:Not Healthy]: 0.6
---------------------
Image: PAT_56_86_479.p

**Good Data**

In [102]:
# Load the data
data_path = r"C:\Users\serru\Downloads\archive (1)\helpme\newdata.csv"
df = pd.read_csv(data_path)

new_df = pd.DataFrame()
new_df['img_id'] = df['image']
new_df['diagnostic'] = df[['MEL', 'BCC', 'AKIEC']].any(axis=1).astype(int)
df = new_df

# Define the function to extract features
def extract_features(folder_path):
    feature_1 = []
    feature_2 = []
    feature_3 = []
    feature_4 = []
    feature_5 = []
    feature_6 = []
    feature_7 = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            original = io.imread(image_path)

            # Ignore the alpha channel (e.g. transparency)
            if original.shape[-1] == 4:
                original = original[..., :3]

            feature_1.append(ef.measure_pigment_network(original))
            feature_2.append(ef.measure_blue_veil(original))
            feature_3.append(ef.measure_vascular(original))
            feature_4.append(ef.measure_globules(original))
            feature_5.append(ef.measure_streaks(original))
            feature_6.append(ef.measure_irregular_pigmentation(original))
            feature_7.append(ef.measure_regression(original))

    return feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7

# Define the folder path for image processing
folder_path_in = r'C:\Users\serru\Downloads\archive (1)\helpme\both_resized'

# Extract features from the images
feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7 = extract_features(folder_path_in)

# Create a DataFrame for the features
features_df = pd.DataFrame()
features_df["img_id"] = [filename for filename in os.listdir(folder_path_in) if filename.endswith(('.jpg', '.png'))]
features_df["1: pigment network"] = feature_1
features_df["2: Blue veil"] = feature_2
features_df["3: Vascular"] = feature_3
features_df["4: Globules"] = feature_4
features_df["5: Streaks"] = feature_5
features_df["6: Pigmentation"] = feature_6
features_df["7: Regression"] = feature_7


features_df['img_id'] = features_df['img_id'].str.replace('.jpg', '')

df_merged3 = pd.merge(df[['img_id', 'diagnostic']], features_df, on='img_id', how='inner')

if df_merged3.shape[0] == 0:
    print("Dataset is empty!")
else:
    # Check the column names
    if 'img_id' not in df_merged3.columns or 'diagnostic' not in df_merged3.columns:
        print("Column names not found in the dataset!")
    else:
        X = df_merged3.drop(['img_id', 'diagnostic'], axis=1)
        Y = df_merged3['diagnostic']

        # Adjust test_size based on the size of your dataset
        test_size = 0.2  # 20% of the data will be used for testing

        # Split the data into training and test sets
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size)

        # Continue with the rest of your code
        classifiers = [LR(), KNN(), DTC()]  # Replace with your trained classifiers

classifiers = [LR(), KNN(), DTC()]  # Replace with your trained classifiers

# Perform cross-validation for each classifier
for classifier in classifiers:
    y_pred = cross_val_predict(classifier, X, Y, cv=5)  # Change cv value as per your requirement
    
    # Calculate evaluation metrics
    f1 = f1_score(Y, y_pred)
    precision = precision_score(Y, y_pred)
    recall = recall_score(Y, y_pred)
    tn, fp, fn, tp = confusion_matrix(Y, y_pred).ravel()
    specificity = tn / (tn + fp)
    
    print(f"Classifier: {type(classifier).__name__}")
    print(f"F1 score: {f1}")
    print(f"Precision: {precision}")
    print(f"Recall/Sensitivity: {recall}")
    print(f"Specificity: {specificity}")
    print("Confusion Matrix:")
    print(confusion_matrix(Y, y_pred))
    print()


Classifier: LogisticRegression
F1 score: 0.8974358974358974
Precision: 0.8641975308641975
Recall/Sensitivity: 0.9333333333333333
Specificity: 0.65625
Confusion Matrix:
[[21 11]
 [ 5 70]]

Classifier: KNeighborsClassifier
F1 score: 0.8227848101265823
Precision: 0.7831325301204819
Recall/Sensitivity: 0.8666666666666667
Specificity: 0.4375
Confusion Matrix:
[[14 18]
 [10 65]]

Classifier: DecisionTreeClassifier
F1 score: 0.7412587412587412
Precision: 0.7794117647058824
Recall/Sensitivity: 0.7066666666666667
Specificity: 0.53125
Confusion Matrix:
[[17 15]
 [22 53]]



**LR MODEL AND PICKLE**

In [103]:
import numpy as np
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score

# Create LR model
lr_model = KNeighborsClassifier(n_neighbors=5)  # Specify the number of neighbors

# Train the LR model on your data (X and Y)
lr_model.fit(X, Y)

# Pickle the LR model
with open(r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\src\Workspace\CleanLast\lr_model.pkl", "wb") as f:
    pickle.dump(lr_model, f)

# Load the pickled LR model
with open(r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\src\Workspace\CleanLast\lr_model.pkl", "rb") as f:
    loaded_lr_model = pickle.load(f)

# Perform cross-validation using the loaded LR model
y_pred = cross_val_predict(loaded_lr_model, X, Y, cv=5)

# Calculate evaluation metrics
f1 = f1_score(Y, y_pred)
precision = precision_score(Y, y_pred)
recall = recall_score(Y, y_pred)
tn, fp, fn, tp = confusion_matrix(Y, y_pred).ravel()
specificity = tn / (tn + fp)

print("Loaded LR Model Metrics:")
print(f"F1 score: {f1}")
print(f"Precision: {precision}")
print(f"Recall/Sensitivity: {recall}")
print(f"Specificity: {specificity}")
print("Confusion Matrix:")
print(confusion_matrix(Y, y_pred))

Loaded KNN Model Metrics:
F1 score: 0.8227848101265823
Precision: 0.7831325301204819
Recall/Sensitivity: 0.8666666666666667
Specificity: 0.4375
Confusion Matrix:
[[14 18]
 [10 65]]


**Predict with LR**

In [104]:
import numpy as np
import pickle
import cv2
import os
import pandas as pd

# Load the pickled LR model
with open(r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\src\Workspace\CleanLast\lr_model.pkl", "rb") as f:
    model = pickle.load(f)

# Load the data
data_path = r'C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\full_data.csv'
df = pd.read_csv(data_path)

# Preprocess the diagnostic column
df['diagnostic'] = df['diagnostic'].map({'BCC': 1, 'MEL': 1, 'SCC': 1, 'ACK': 0, 'NEV': 0, 'SEK': 0})

# Define the function to extract features
def extract_features2(image_path):
    image = cv2.imread(image_path)
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # Perform feature extraction operations
    feature_1 = ef.measure_pigment_network(lab_image)
    feature_2 = ef.measure_blue_veil(lab_image)
    feature_3 = ef.measure_vascular(lab_image)
    feature_4 = ef.measure_globules(lab_image)
    feature_5 = ef.measure_streaks(lab_image)
    feature_6 = ef.measure_irregular_pigmentation(lab_image)
    feature_7 = ef.measure_regression(lab_image)

    return feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7

# Define the folder path for image processing
folder_path_in = r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\ColorMask\Test"  # Replace with the path to the folder containing images

# Iterate over images in the folder
for filename in os.listdir(folder_path_in):
    if filename.endswith(".jpg") or filename.endswith(".png"):  # Update the file extensions if necessary
        image_path = os.path.join(folder_path_in, filename)
        
        # Extract features from the image
        feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7 = extract_features2(image_path)
        
        # Prepare the data for prediction
        X = np.array([feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7]).reshape(1, -1)
        
        # Make predictions using the loaded LR model
        probabilities = model.predict_proba(X)
        
        # Get the probability for class 1 (positive class)
        probability_positive = probabilities[0][1]
        
        # Map the probability to a value between 0 and 1
        mapped_value = 1.0 - probability_positive  # Invert the probability to get the desired mapping

        # Get the diagnosis from the df DataFrame using the modified filename
        diagnosis = df.loc[df['img_id'] == filename, 'diagnostic'].values[0]

        # Map the diagnosis to a human-readable label
        diagnosis_label = 'Not Healthy' if diagnosis == 1 else 'Healthy'
        
        # Print the image filename, diagnosis, and mapped value
        print("Image:", filename)
        print("Diagnosis from csv:", diagnosis_label)
        print("Prediction [0:Healthy - 1:Not Healthy]:", mapped_value)
        print("---------------------")


Image: PAT_109_868_723.png
Diagnosis from csv: Not Healthy
Prediction [0:Healthy - 1:Not Healthy]: 0.0
---------------------
Image: PAT_236_361_180.png
Diagnosis from csv: Healthy
Prediction [0:Healthy - 1:Not Healthy]: 0.0
---------------------
Image: PAT_26_37_865.png
Diagnosis from csv: Healthy
Prediction [0:Healthy - 1:Not Healthy]: 0.0
---------------------
Image: PAT_270_417_257.png
Diagnosis from csv: Not Healthy
Prediction [0:Healthy - 1:Not Healthy]: 0.19999999999999996
---------------------
Image: PAT_333_1469_499.png
Diagnosis from csv: Healthy
Prediction [0:Healthy - 1:Not Healthy]: 0.0
---------------------
Image: PAT_38_54_234.png
Diagnosis from csv: Not Healthy
Prediction [0:Healthy - 1:Not Healthy]: 0.0
---------------------
Image: PAT_398_799_163.png
Diagnosis from csv: Healthy
Prediction [0:Healthy - 1:Not Healthy]: 0.0
---------------------
Image: PAT_42_58_13.png
Diagnosis from csv: Not Healthy
Prediction [0:Healthy - 1:Not Healthy]: 0.0
---------------------
Image:

**Mixed data**

In [116]:
df_merged4 = pd.concat([df_merged3, df_merged2], ignore_index=True)
df_merged4

,img_id,diagnostic,1: pigment network,2: Blue veil,3: Vascular,4: Globules,5: Streaks,6: Pigmentation,7: Regression
0,ISIC_0024306,0,22.113037,21976,21940,128,1.273240,29.417419,0
1,ISIC_0024307,0,26.664734,26870,23634,101,1.273240,40.980530,0
2,ISIC_0024308,0,23.535156,20676,23280,213,1.273240,28.671265,0
3,ISIC_0024309,0,22.734070,22935,22089,82,2.298106,34.982300,0
4,ISIC_0024310,1,15.336609,8397,12758,207,1.273240,22.416687,0
...,...,...,...,...,...,...,...,...,...
207,PAT_1730_3236_201.png,0,4.988098,4007,74,86,1.273240,7.115173,42
208,PAT_1765_3337_365.png,0,5.062866,4753,935,85,1.273240,6.770325,162
209,PAT_1942_3918_497.png,0,2.795410,29,2657,29,1.273240,3.939819,0
210,PAT_2056_4364_112.png,0,2.391052,1846,9,31,1.273240,3.442383,0


In [117]:
# Define the function to extract features
def extract_features(folder_path):
    feature_1 = []
    feature_2 = []
    feature_3 = []
    feature_4 = []
    feature_5 = []
    feature_6 = []
    feature_7 = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            original = io.imread(image_path)

            # Ignore the alpha channel (e.g. transparency)
            if original.shape[-1] == 4:
                original = original[..., :3]

            feature_1.append(ef.measure_pigment_network(original))
            feature_2.append(ef.measure_blue_veil(original))
            feature_3.append(ef.measure_vascular(original))
            feature_4.append(ef.measure_globules(original))
            feature_5.append(ef.measure_streaks(original))
            feature_6.append(ef.measure_irregular_pigmentation(original))
            feature_7.append(ef.measure_regression(original))

    return feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7

# Define the folder path for image processing
folder_path_in = r'C:\Users\serru\Downloads\archive (1)\mixed'

# Extract features from the images
feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7 = extract_features(folder_path_in)

# Create a DataFrame for the features
features_df = pd.DataFrame()
features_df["img_id"] = [filename for filename in os.listdir(folder_path_in) if filename.endswith(('.jpg', '.png'))]
features_df["1: pigment network"] = feature_1
features_df["2: Blue veil"] = feature_2
features_df["3: Vascular"] = feature_3
features_df["4: Globules"] = feature_4
features_df["5: Streaks"] = feature_5
features_df["6: Pigmentation"] = feature_6
features_df["7: Regression"] = feature_7

# Split the data into training and testing sets
X = df_merged4.drop(['img_id', 'diagnostic'], axis=1)
Y = df_merged4['diagnostic']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5)

classifiers = [LR(), KNN(), DTC()]  # Replace with your trained classifiers

# Perform cross-validation for each classifier
for classifier in classifiers:
    y_pred = cross_val_predict(classifier, X, Y, cv=5)  # Change cv value as per your requirement
    
    # Calculate evaluation metrics
    f1 = f1_score(Y, y_pred)
    precision = precision_score(Y, y_pred)
    recall = recall_score(Y, y_pred)
    tn, fp, fn, tp = confusion_matrix(Y, y_pred).ravel()
    specificity = tn / (tn + fp)
    
    print(f"Classifier: {type(classifier).__name__}")
    print(f"F1 score: {f1}")
    print(f"Precision: {precision}")
    print(f"Recall/Sensitivity: {recall}")
    print(f"Specificity: {specificity}")
    print("Confusion Matrix:")
    print(confusion_matrix(Y, y_pred))
    print()

Classifier: LogisticRegression
F1 score: 0.7114093959731543
Precision: 0.6198830409356725
Recall/Sensitivity: 0.8346456692913385
Specificity: 0.23529411764705882
Confusion Matrix:
[[ 20  65]
 [ 21 106]]

Classifier: KNeighborsClassifier
F1 score: 0.6716417910447762
Precision: 0.6382978723404256
Recall/Sensitivity: 0.7086614173228346
Specificity: 0.4
Confusion Matrix:
[[34 51]
 [37 90]]

Classifier: DecisionTreeClassifier
F1 score: 0.62015503875969
Precision: 0.6106870229007634
Recall/Sensitivity: 0.6299212598425197
Specificity: 0.4
Confusion Matrix:
[[34 51]
 [47 80]]



**AND PICKLE

In [119]:
import numpy as np
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score, confusion_matrix, precision_score, recall_score

# Create LR model
model = KNeighborsClassifier(n_neighbors=5)  # Specify the number of neighbors

# Train the LR model on your data (X and Y)
model.fit(X, Y)

# Pickle the LR model
with open(r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\src\Workspace\CleanLast\lr_mixed_model.pkl", "wb") as f:
    pickle.dump(model, f)

# Load the pickled LR model
with open(r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\src\Workspace\CleanLast\lr_mixed_model.pkl", "rb") as f:
    loaded_model = pickle.load(f)

# Perform cross-validation using the loaded KNN model
y_pred = cross_val_predict(loaded_lr_model, X, Y, cv=5)

# Calculate evaluation metrics
f1 = f1_score(Y, y_pred)
precision = precision_score(Y, y_pred)
recall = recall_score(Y, y_pred)
tn, fp, fn, tp = confusion_matrix(Y, y_pred).ravel()
specificity = tn / (tn + fp)

print("Loaded Mixed LR Model Metrics:")
print(f"F1 score: {f1}")
print(f"Precision: {precision}")
print(f"Recall/Sensitivity: {recall}")
print(f"Specificity: {specificity}")
print("Confusion Matrix:")
print(confusion_matrix(Y, y_pred))

Loaded Mixed LR Model Metrics:
F1 score: 0.6716417910447762
Precision: 0.6382978723404256
Recall/Sensitivity: 0.7086614173228346
Specificity: 0.4
Confusion Matrix:
[[34 51]
 [37 90]]


**Predict with LR mixed**

In [120]:
import numpy as np
import pickle
import cv2
import os
import pandas as pd

# Load the pickled LR model
with open(r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\src\Workspace\CleanLast\lr_mixed_model.pkl", "rb") as f:
    model = pickle.load(f)

# Load the data
data_path = r'C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\full_data.csv'
df = pd.read_csv(data_path)

# Preprocess the diagnostic column
df['diagnostic'] = df['diagnostic'].map({'BCC': 1, 'MEL': 1, 'SCC': 1, 'ACK': 0, 'NEV': 0, 'SEK': 0})

# Define the function to extract features
def extract_features2(image_path):
    image = cv2.imread(image_path)
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)

    # Perform feature extraction operations
    feature_1 = ef.measure_pigment_network(lab_image)
    feature_2 = ef.measure_blue_veil(lab_image)
    feature_3 = ef.measure_vascular(lab_image)
    feature_4 = ef.measure_globules(lab_image)
    feature_5 = ef.measure_streaks(lab_image)
    feature_6 = ef.measure_irregular_pigmentation(lab_image)
    feature_7 = ef.measure_regression(lab_image)

    return feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7

# Define the folder path for image processing
folder_path_in = r"C:\Users\serru\OneDrive\Documents\Project2\Project-2-Medical-Imaging\data\ColorMask\Test"  # Replace with the path to the folder containing images

# Iterate over images in the folder
for filename in os.listdir(folder_path_in):
    if filename.endswith(".jpg") or filename.endswith(".png"):  # Update the file extensions if necessary
        image_path = os.path.join(folder_path_in, filename)
        
        # Extract features from the image
        feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7 = extract_features2(image_path)
        
        # Prepare the data for prediction
        X = np.array([feature_1, feature_2, feature_3, feature_4, feature_5, feature_6, feature_7]).reshape(1, -1)
        
        # Make predictions using the loaded LR model
        probabilities = model.predict_proba(X)
        
        # Get the probability for class 1 (positive class)
        probability_positive = probabilities[0][1]
        
        # Map the probability to a value between 0 and 1
        mapped_value = 1.0 - probability_positive  # Invert the probability to get the desired mapping

        # Get the diagnosis from the df DataFrame using the modified filename
        diagnosis = df.loc[df['img_id'] == filename, 'diagnostic'].values[0]

        # Map the diagnosis to a human-readable label
        diagnosis_label = 'Not Healthy' if diagnosis == 1 else 'Healthy'
        
        # Print the image filename, diagnosis, and mapped value
        print("Image:", filename)
        print("Diagnosis from csv:", diagnosis_label)
        print("Prediction [0:Healthy - 1:Not Healthy]:", mapped_value)
        print("---------------------")


Image: PAT_109_868_723.png
Diagnosis from csv: Not Healthy
Prediction [0:Healthy - 1:Not Healthy]: 1.0
---------------------
Image: PAT_236_361_180.png
Diagnosis from csv: Healthy
Prediction [0:Healthy - 1:Not Healthy]: 0.19999999999999996
---------------------
Image: PAT_26_37_865.png
Diagnosis from csv: Healthy
Prediction [0:Healthy - 1:Not Healthy]: 0.8
---------------------
Image: PAT_270_417_257.png
Diagnosis from csv: Not Healthy
Prediction [0:Healthy - 1:Not Healthy]: 0.4
---------------------
Image: PAT_333_1469_499.png
Diagnosis from csv: Healthy
Prediction [0:Healthy - 1:Not Healthy]: 0.8
---------------------
Image: PAT_38_54_234.png
Diagnosis from csv: Not Healthy
Prediction [0:Healthy - 1:Not Healthy]: 0.8
---------------------
Image: PAT_398_799_163.png
Diagnosis from csv: Healthy
Prediction [0:Healthy - 1:Not Healthy]: 0.4
---------------------
Image: PAT_42_58_13.png
Diagnosis from csv: Not Healthy
Prediction [0:Healthy - 1:Not Healthy]: 0.0
---------------------
Image: